# source_code_2.ipynb

This Jupyter notebook produces some figures for:  
Verduzco-Flores S., De Schutter E. (2021) "Adaptive plasticity in the spinal cord can produce reaching from scratch and reproduces motor cortex directional tuning."

The simulations done in this notebook are for an isolated spinal cord.

Please make sure that the `%cd ...` statement below changes the folder to the
location of your Draculab installation.

In [ ]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
from multiprocessing import Pool

In [ ]:
%load_ext Cython

In [ ]:
%%cython
from draculab import *
from plants import spinal_plants

In [ ]:
# load a parameter configuration
ps1={'A__M_lrate': 20.0,
     'A__M_w_max_frac': 0.34,
     'A__M_w_sum': 1.0,
     'AL_thresh': 0.56,
     'b_e': 1.,
     'C__C_antag': 1.6,
     'C__C_p_antag': 0.15,
     'C__C_p_syne': 0.26,
     'C__C_syne': 1.1,
     'C_adapt_amp': 0.0,
     'C_cid': 0.17,
     'C_sigma': 0.5,
     'C_slope': 2.25,
     'C_tau': 0.24,
     'C_tau_slow': 2.0,
     'C_thresh': 1.14,
     'CE__CI_w': 0.39,
     'CI__CE_w': -1.8,
     'g_e_03': 20.,
     'CI_slope': 3.9,
     'CI_tau': 0.06,
     'CI_thresh': 1.37,
     'g_e_factor': 3.,
     'II_g_03': 2.73,
     'M__C_lrate': 500.0,
     'M__C_w_sum': 3.28,
     'M__M_w': 0.0,
     'M_cid': 1.,
     'M_des_out_w_abs_sum': 1.87,
     'M_tau': 0.012,
     'SF_thresh_03': 0.59,
     'SPF__SPF_w': -1.6,
     'fitness': None,
     'n_evals': 0,
     't_pres': 40.,
     'par_heter': .01}
    
cfg = ps1
C_noise = False # whether C units are noisy (use euler_maru integrator)
par_heter = cfg['par_heter']

In [ ]:
def create_net(coords):
    """ Create a network with a hand in the given coordinates.
    
    Args:
        coords: list-like with [x,y] coordinates.
        
    Returns:
        net: a network where the initial hand position is the given coordinates.
        pops_dict: a dictionary with the names and unit IDs for all populations. 
    """

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # network and plant parameters
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    net_params = {'min_delay' : 0.005,
                  'min_buff_size' : 8 }
    # Find the angles corresponding to the given coordinates
    L1 =0.3  # upper arm length to create initial angles
    L2 = 0.3 # forearm length
    # copied from coords_to_angs
    x = coords[0]
    y = coords[1]
    Rsq = x*x + y*y
    R = np.sqrt(Rsq)
    if R > L1 + L2:
        raise ValueError('Unreachable coordinate given')
    qe = np.pi - np.arccos((L1**2 + L2**2 - Rsq) / (2.*L1*L2))
    qs = np.arctan2(y,x) - np.arcsin((L2/R)*np.sin(qe))
    P_params = {'type' : plant_models.bouncy_planar_arm_v3,
              'mass1': 1.,
              'mass2': 1.,
              's_min' : -0.8,
              'p1' : (-0.01, 0.04),
              'p2' : (0.29, 0.03),
              'p3' : (0., 0.05),
              'p4' : (0.1, 0.04),
              'p5' : (0.01, -0.05),
              'p6' : (0.11, -0.04),
              'p10': (0.29, 0.03),
              'init_q1': qs,
              'init_q2': qe,
              'init_q1p': 0.,
              'init_q2p': 0.,
              'g': 0.0,
              'mu1': 3.,
              'mu2': 3.,
              'l_torque' : 0.01,
              'l_visco' : 0.01,
              'g_e' : cfg['g_e_factor']*np.array([cfg['g_e_03'], 20., 20.,
                                                  cfg['g_e_03'], 22., 23.]),
              'l0_e' : [1.]*6,
              'Ia_gain' : 2.5*np.array([3.,10.,10., 3.,10.,10.]),
              'II_gain' : 2.*np.array([cfg['II_g_03'], 8., 8.,
                                       cfg['II_g_03'], 8., 8.]),
              'Ib_gain' : 1.,
              'T_0' : 10.,
              'k_pe_e' : 20.,  #8
              'k_se_e' : 20., #13
              'b_e' : cfg['b_e'],
              'g_s' : 0.02,
              'k_pe_s' : 2., 
              'k_se_s' : 2.,
              'g_d' : 0.01,
              'k_pe_d' : .2, #.1,
              'k_se_d' : 1., #2.,
              'b_s' : .5,
              'b_d' : 2.,#3.,
              'l0_s': .7,
              'l0_d': .8,
              'fs' : 0.1,
              'se_II' : 0.5,
              'cd' : 0.5,
              'cs' : 0.5,
              'tau' : 0.1   # ficticious time constant used in create_freqs_steps
               }
    
    net = network(net_params) # create network

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # UNIT PARAMETER DICTIONARIES
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    randz = lambda n: (1. + par_heter*(np.random.rand(n)-0.5))

    AL_params = {'type' : unit_types.sigmoidal,
                 'thresh' : cfg['AL_thresh'] * randz(6),
                 'slope' : 2. * randz(6),
                 'init_val' : 0.1 * randz(6),
                 'tau' : 0.02 * randz(6) }
    CE_params = {'type' : unit_types.rga_adapt_sig,
                'integ_meth' : 'euler_maru' if C_noise else 'odeint',
                'init_val' : 0.2*randz(6), #[r*np.array([0.2]) for r in np.random.random(6)],
                'multidim' : False,
                'slope' : cfg['C_slope'],
                'thresh' : cfg['C_thresh'],
                'tau' : cfg['C_tau'],
                'tau_fast': 0.01,
                'tau_mid' : 0.05,
                'tau_slow' : cfg['C_tau_slow'],
                'custom_inp_del' : int(round(cfg['C_cid']/net_params['min_delay'])),
                'delay' : 0.31, # to be set below
                'adapt_amp' : cfg['C_adapt_amp'],
                'mu' : 0.,
                'sigma' : cfg['C_sigma'] }
    CI_params = {'type' : unit_types.rga_adapt_sig,
                'integ_meth' : 'euler_maru' if C_noise else 'odeint',
                'init_val' : 0.2*randz(6), #[r*np.array([0.2]) for r in np.random.random(6)],
                'multidim' : False,
                'slope' : cfg['CI_slope'] if 'CI_slope' in cfg else cfg['C_slope'],
                'thresh' : cfg['CI_thresh'] if 'CI_thresh' in cfg else cfg['C_thresh'],
                'tau' :  cfg['CI_tau'] if 'CI_tau' in cfg else cfg['C_tau'],
                'tau_fast': 0.01,
                'tau_mid' : 0.05,
                'tau_slow' : cfg['C_tau_slow'],
                'custom_inp_del' : int(round(cfg['C_cid']/net_params['min_delay'])),
                'delay' : 0.31, # to be set below
                'adapt_amp' : cfg['C_adapt_amp'],
                'mu' : 0.,
                'sigma' : cfg['C_sigma'] }
    INP_params = {'type' : unit_types.source,
                  'init_val' : 0.5,
                  'tau_fast' : 0.02,
                  'tau_mid' : 0.1,
                  'function' : lambda t: None }
    track_params = {'type' : unit_types.source,
                    'init_val' : 0.02,
                    'function' : lambda t: None }

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CONNECTION DICTIONARIES
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # AL to P ------------------------------------------------
    AL__P_conn = {'inp_ports' : list(range(6)),
                 'delays': 0.02 }
    AL__P_syn = {'type': synapse_types.static,
                'init_w' : 1. }
    # CE, CI to AL ----------------------------------------------
    CE__AL_conn = {'rule' : 'one_to_one',
                   'delay' : 0.01 }
    CE__AL_syn = {'type' : synapse_types.static,
                  'init_w' : 1. }
    CI__AL_conn = {'rule' : 'one_to_one',
                   'delay' : 0.01 }
    CI__AL_syn = {'type' : synapse_types.static,
                  'init_w' : -1. }
    # CE,CI to CE,CI within the same population -----------------
    CE__CI_conn = {'rule' : 'one_to_one',
                   'delay' : 0.01 }
    CI__CE_conn = {'rule' : 'one_to_one',
                   'delay' : 0.01 }
    CE__CI_syn = {'type' : synapse_types.static,
                  'inp_ports' : 1,
                  'init_w' : cfg['CE__CI_w'] }
    CI__CE_syn = {'type' : synapse_types.static, #static, #corr_inh,
                  'inp_ports' : 1,
                  'lrate' : .0,
                  'des_act' : 0.5,
                  'init_w' : cfg['CI__CE_w'] }  
    # synergist and antagonist connections in C
    C__C_conn = {'rule': 'one_to_one',
                 'allow_autapses' : False,
                 'delay' : 0.01 }
    C__C_syn_antag = {'type' : synapse_types.static,
                      'inp_ports': 1, # "lateral" port of rga_21 synapses
                      'init_w' : cfg['C__C_antag'] }
    C__C_syn_p_antag = {'type' : synapse_types.static,
                        'inp_ports': 1,
                        'init_w' : cfg['C__C_p_antag'] }
    C__C_syn_syne = {'type' : synapse_types.static,
                      'inp_ports': 1,
                      'init_w' : cfg['C__C_syne'] }
    C__C_syn_p_syne = {'type' : synapse_types.static,
                      'inp_ports': 1,
                      'init_w' : cfg['C__C_p_syne']}
    C__C_syn_null_lat = {'type' : synapse_types.static, # connection with static weight zero
                       'inp_ports': 1,
                       'init_w' : 0. }
    # INP to CE
    INP_CE_conn = {'rule' : 'all_to_all',
                'delay' : 0.01 }
    INP_CE_syn = {'type' : synapse_types.static,
                 'inp_ports': 0,
                 'init_w' : 0.0 } # these are changed by get_dir_field()
    INP_CI_conn = {'rule' : 'all_to_all',
                'delay' : 0.01 }
    INP_CI_syn = {'type' : synapse_types.static,
                 'inp_ports': 0,
                 'init_w' : 0.0 } # these are changed by get_dir_field()

    #*************************************************************

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CREATING UNITS
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    AL = net.create(6, AL_params)
    CE = net.create(6, CE_params)
    CI = net.create(6, CI_params)
    INP = net.create(1, INP_params)
    P = net.create(1, P_params)

    ipx_track = net.create(12, track_params) # x coordinates of insertion points
    ipy_track = net.create(12, track_params) # y coordinates of insertion points

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CONNECTING
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # from CE to AL
    net.connect(CE, AL, CE__AL_conn, CE__AL_syn)
    # from CI to AL
    net.connect(CI, AL, CI__AL_conn, CI__AL_syn)
    # from AL to P
    net.set_plant_inputs(AL, P, AL__P_conn, AL__P_syn)
    # intraspinal connections
    # We organize the spinal connections through 4 types of symmetric relations
    # these lists are used to set intraspinal connections and test connection matrices
    antagonists = [(0,3), (1,2), (4,5)]
    part_antag = [(0,2),(0,5), (3,4), (1,3)]
    synergists = [(0,1), (0,4), (2,3), (3,5)]
    part_syne = [(1,4), (2,5)]
    self_conn = [(x,x) for x in range(6)]
    # adding the symmetric relation (inverting source and target)
    antagonists += [(p[1],p[0]) for p in antagonists]
    part_antag += [(p[1],p[0]) for p in part_antag]
    synergists += [(p[1],p[0]) for p in synergists]
    part_syne += [(p[1],p[0]) for p in part_syne]
    all_pairs = [(i,j) for i in range(6) for j in range(6)]
    
    for pair in all_pairs:
        if pair in synergists:
            net.connect([CE[pair[0]]], [CE[pair[1]]], C__C_conn, C__C_syn_syne)
        elif pair in part_syne:
            net.connect([CE[pair[0]]], [CE[pair[1]]], C__C_conn, C__C_syn_p_syne)
        elif pair in antagonists:
            net.connect([CE[pair[0]]], [CI[pair[1]]], C__C_conn, C__C_syn_antag)
        elif pair in part_antag:
            net.connect([CE[pair[0]]], [CI[pair[1]]], C__C_conn, C__C_syn_p_antag)
        elif pair in self_conn:
            net.connect([CE[pair[0]]], [CI[pair[1]]], CE__CI_conn, CE__CI_syn)
            net.connect([CI[pair[0]]], [CE[pair[1]]], CI__CE_conn, CI__CE_syn)       
    
    # from INP to CE,CI
    net.connect(INP, CE, INP_CE_conn, INP_CE_syn)
    net.connect(INP, CI, INP_CI_conn, INP_CI_syn)
    
    # SETTING THE FUNCTION FOR INP
    net.units[INP[0]].set_function(lambda t: 1. ) # if t > 0.1 and t < 6. else 0.)

    # TRACKING OF INSERTION POINTS (for the arm animation)
    # make the source units track the tensions
    def create_xtracker(arm_id, idx):
        return lambda t: net.plants[arm_id].ip[idx][0]
    def create_ytracker(arm_id, idx):
        return lambda t: net.plants[arm_id].ip[idx][1]
    for idx, uid in enumerate(ipx_track):
        net.units[uid].set_function(create_xtracker(P, idx))
    for idx, uid in enumerate(ipy_track):
        net.units[uid].set_function(create_ytracker(P, idx))
        
    # TEST OF ARM LENGTHS
    l1 = net.plants[0].l_arm # length of upper arm
    l2 = net.plants[0].l_farm # length of forearm
    assert (l1 == L1) and (l2 == L2), "L1, L2 arm lengths are inconsistent"

    pops_list = [AL, CE, CI, INP, P, ipx_track, ipy_track]
    pops_names = ['AL','CE', 'CI', 'INP', 'P', 'ipx_track', 'ipy_track']
    pops_dict = {pops_names[idx] : pops_list[idx] for idx in range(len(pops_names))}
    return net, pops_dict

In [ ]:
# A method to obtain a direction field for given stimulated populations
def get_dir_field(stim_pops, sim_time, CE_w=1., CI_w=1.):
    """ Given a list of spinal units to stimulate, return a direction field.
    
        Args:
            stim_pops: a list with the indexes of CE/CI units to stimulate.
            sim_time: time to run the simulations.
            CE_w: synaptic weight for connections from input to CE (default 1)
            CI_w: synaptic weight for connections from input to CI (default 1)
            
        Returns:
            A 5-tuple with these arrays, in this order:
                hand_coords: each row contains initial hand coordinates
                final_hand_coordinates: each row contains corresponding final hand coordinates.
                times: numpy array with the times for the data arrays in `all_data`, `all_plant_data`
                all_data: list with a 'data' array for every hand position in hand_coords.
                all_plant_data: list with a 'plant_data' array for each hand position.
    """
    #1) Choose a grid of initial hand coordinates
    min_s_ang = -0.4 # minimum shoulder angle
    max_s_ang = 1.2  # maximum shoulder angle
    min_e_ang = 0.1 # minimum elbow angle
    max_e_ang = 2.5 # maximum elbow angle
    s_subs = 10 # number of shoulder angle subdivisions in grid
    e_subs = 10 # number of elbow angle subdivisions in grid
    n_coords = s_subs * e_subs # number of coordinates to generate
    net, pops_dict = create_net([0.3,0.3]) # create a reference network
    l_arm = net.plants[0].l_arm # length of upper arm
    l_farm = net.plants[0].l_farm # length of forearm
    hand_coords = [[0.,0.] for _ in range(n_coords)]
    s_angs = np.linspace(min_s_ang, max_s_ang, s_subs)
    e_angs = np.linspace(min_e_ang, max_e_ang, s_subs)
    #s_angs = (np.random.random(n_coords)+min_s_ang)*(max_s_ang-min_s_ang)
    #e_angs = (np.random.random(n_coords)+min_e_ang)*(max_e_ang-min_e_ang)
    all_s_angs = np.tile(s_angs,(e_subs,1)).transpose().flatten()
    all_e_angs = np.tile(e_angs,(1,s_subs)).flatten()
    for i in range(n_coords):
        hand_coords[i][0] = l_arm*np.cos(all_s_angs[i]) + l_farm*np.cos(all_s_angs[i]+all_e_angs[i]) # x-coordinate
        hand_coords[i][1] = l_arm*np.sin(all_s_angs[i]) + l_farm*np.sin(all_s_angs[i]+all_e_angs[i]) # y-coordinate
    hand_coords = np.array(hand_coords)
  
    # 2) Run a simulation for each grid point
    all_data = []
    all_plant_data = []
    final_hand_coords = np.zeros_like(hand_coords)
    for c_idx, hcoords in enumerate(hand_coords):
        # 2.1) Initialize
        net, pops_dict = create_net(hcoords)
        CE = pops_dict['CE']
        CI = pops_dict['CI']
        #P = pops_dict['P']
        INP = pops_dict['INP']
        # 2.1.1) Set inputs to spinal units
        CE_stim = [CE[i] for i in stim_pops] # list of ID's of CE units to stimulate
        CI_stim = [CI[i] for i in stim_pops] # list of ID's of CI units to stimulate
        for ce in CE_stim:
            for syn in net.syns[ce]:
                if syn.preID == INP[0]:
                    syn.w = CE_w
        for ci in CI_stim:
            for syn in net.syns[ci]:
                if syn.preID == INP[0]:
                    syn.w = CI_w
        # 2.2) Run sim, save final hand coordinates
        start_time = time.time()
        times, data, plant_data  = net.flat_run(sim_time)
        #times, data, plant_data  = net.run(sim_time)
        print('sim %d: %.2f s' % (c_idx, time.time() - start_time),end=", ")
        if (c_idx+1) % 5 == 0: print("...")
        ## converting lists to arrays, and reducing size
        data = np.array(data); data = data[:,::10]; 
        plant_data = np.array(plant_data); plant_data = plant_data[0][::10,:]
        all_data.append(data)
        all_plant_data.append(plant_data)

        # 2.3) Extract final hand locations
            # angles of shoulder and elbow
        theta_s = plant_data[-1,0]
        theta_e = plant_data[-1,2]
        phi = theta_s + theta_e # elbow angle wrt x axis
            # coordinates of hand and elbow
        xe = np.cos(theta_s)*l_arm
        ye = np.sin(theta_s)*l_arm
        xh = xe + np.cos(phi)*l_farm
        yh = ye + np.sin(phi)*l_farm
        final_hand_coords[c_idx,:] = np.array([xh, yh])
    print("\n-----------------------------------------------")
        
    return (hand_coords, final_hand_coords, times[::10], all_data, all_plant_data)


In [ ]:
# A parallel method to obtain a direction field for given stimulated populations

def run_coords(arg):
    """ Given hand coordinates, run a network with the arm initialized at those coordinates.
    
        Args: 4-tuple with these entries:
            hcoords: list-like with 2 elements, corresponding to x, y hand coordinates
            stim_pops: list with populations to stimulate
            CE_w: weight of connections from INP to CE
            CI_w: weight of connections from INP to CI
            
        Returns: A 3-tuple with these elements
            data: unit data array returned after the simulation.
            plant_data: plant data array returned after the simulation.
            final_hand_coords: coordinates of the hand at the end of the simulation.
            
    """
    hcoords = arg[0]
    stim_pops = arg[1]
    CE_w = arg[2]
    CI_w = arg[3]
    net, pops_dict = create_net(hcoords)
    CE = pops_dict['CE']
    CI = pops_dict['CI']
    INP = pops_dict['INP']
    # 2.1.1) Set inputs to spinal units
    CE_stim = [CE[i] for i in stim_pops] # list of ID's of CE units to stimulate
    CI_stim = [CI[i] for i in stim_pops] # list of ID's of CI units to stimulate
    for ce in CE_stim:
        for syn in net.syns[ce]:
            if syn.preID == INP[0]:
                syn.w = CE_w
    for ci in CI_stim:
        for syn in net.syns[ci]:
            if syn.preID == INP[0]:
                syn.w = CI_w
    # 2.2) Run sim, save final hand coordinates
    
    times, data, plant_data  = net.flat_run(sim_time)
    #times, data, plant_data  = net.run(sim_time)
    ## converting lists to arrays, and reducing size
    data = np.array(data); data = data[:,::10]; 
    plant_data = np.array(plant_data); plant_data = plant_data[0][::10,:]
    # 2.3) Extract final hand locations
        # angles of shoulder and elbow
    theta_s = plant_data[-1,0]
    theta_e = plant_data[-1,2]
    phi = theta_s + theta_e # elbow angle wrt x axis
        # coordinates of hand and elbow
    xe = np.cos(theta_s)*net.plants[0].l_arm
    ye = np.sin(theta_s)*net.plants[0].l_arm
    xh = xe + np.cos(phi)*net.plants[0].l_farm
    yh = ye + np.sin(phi)*net.plants[0].l_farm
    
    return (data, plant_data, np.array([xh, yh])) 

def get_dir_field_par(stim_pops, sim_time, CE_w=1., CI_w=1., n_procs=4):
    """ Given a list of spinal units to stimulate, return a direction field.
    
        Parallel version.
        
        Args:
            stim_pops: a list with the indexes of CE/CI units to stimulate.
            sim_time: time to run the simulations.
            CE_w: synaptic weight for connections from input to CE (default 1)
            CI_w: synaptic weight for connections from input to CI (default 1)
            n_procs: number of processes to use.
            
        Returns:
            A 5-tuple with these arrays, in this order:
                hand_coords: each row contains initial hand coordinates
                final_hand_coordinates: each row contains corresponding final hand coordinates.
                times: numpy array with the times for the data arrays in `all_data`, `all_plant_data`
                all_data: list with a 'data' array for every hand position in hand_coords.
                all_plant_data: list with a 'plant_data' array for each hand position.
    """
    #1) Choose a grid of initial hand coordinates
    min_s_ang = -0.4 # minimum shoulder angle
    max_s_ang = 1.2  # maximum shoulder angle
    min_e_ang = 0.1 # minimum elbow angle
    max_e_ang = 2.5 # maximum elbow angle
    s_subs = 30 # number of shoulder angle subdivisions in grid
    e_subs = 30 # number of elbow angle subdivisions in grid
    n_coords = s_subs * e_subs # number of coordinates to generate
    net, pops_dict = create_net([0.3,0.3]) # create a reference network
    l_arm = net.plants[0].l_arm # length of upper arm
    l_farm = net.plants[0].l_farm # length of forearm
    hand_coords = [[0.,0.] for _ in range(n_coords)]
    s_angs = np.linspace(min_s_ang, max_s_ang, s_subs)
    e_angs = np.linspace(min_e_ang, max_e_ang, s_subs)
    #s_angs = (np.random.random(n_coords)+min_s_ang)*(max_s_ang-min_s_ang)
    #e_angs = (np.random.random(n_coords)+min_e_ang)*(max_e_ang-min_e_ang)
    all_s_angs = np.tile(s_angs,(e_subs,1)).transpose().flatten()
    all_e_angs = np.tile(e_angs,(1,s_subs)).flatten()
    for i in range(n_coords):
        hand_coords[i][0] = l_arm*np.cos(all_s_angs[i]) + l_farm*np.cos(all_s_angs[i]+all_e_angs[i]) # x-coordinate
        hand_coords[i][1] = l_arm*np.sin(all_s_angs[i]) + l_farm*np.sin(all_s_angs[i]+all_e_angs[i]) # y-coordinate
    hand_coords = np.array(hand_coords)
  
    # 2) Run a simulation for each grid point
    args = [(hcoords, stim_pops, CE_w, CI_w) for hcoords in hand_coords]
    
    with Pool(n_procs) as p:
        results = list(p.map(run_coords, args))
        p.close()
        p.join()
    
    all_data = [res[0] for res in results]
    all_plant_data = [res[1] for res in results]
    final_hand_coords = [res[2] for res in results]
    times = np.linspace(0., sim_time, all_data[0].shape[1])
        
    return (hand_coords, final_hand_coords, times, all_data, all_plant_data)


If using the next cell to run parallel simulations, make sure to set the number of processes (`n_procs`) according to your computer's capability.

In [ ]:
# run simulations for all combinations
sim_time = .5 # simulation time
E_amp = 8. # amplitude of stimulus to CE population
I_amp = 8. # amplitude of stimulus to CI population
n_procs = 30 # number of processes to use for grid simulation

all_hc = [] # all grids of initial hand coordinates
all_fhc = [] # all grids of final hand coordinates
all_t = [] # all time vectors
all_ad = [] # all grids of unit simulation data
all_apd = []# all grids of plant simulation data

# define CE, CI elements to be stimulated
single_stim = [[i] for i in (range(6))]
# all combinations of two different populations:
double_stim = []
for i in range(6):
    for j in range(i+1,6):
        double_stim.append([i,j])

all_stim = [[]] + single_stim + double_stim

start_wctime = time.time()
for stim in all_stim:
    hc, fhc, t, ad, apd = get_dir_field_par(stim, sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs)
    all_hc.append(hc)
    all_fhc.append(fhc)
    all_t.append(t)
    all_ad.append(ad)
    all_apd.append(apd)
    print(".", end='')
    
print('\n Execution time is %s seconds' % (time.time() - start_wctime))

In [ ]:
# For each pair of muscles, get the average angle between fields (a,b) and a+b
rem_rf = False # whether to remove the resting field
all_avg_angs = []
if rem_rf:
    dir0 = all_fhc[0] - all_hc[0] # resting direction field
else:
    dir0 = 0.
for idx, pair in enumerate(double_stim):
    dirs_a = all_fhc[1+pair[0]] - all_hc[1+pair[0]] - dir0
    dirs_b = all_fhc[1+pair[1]] - all_hc[1+pair[1]] - dir0
    dirs_ab = all_fhc[7+idx] - all_hc[7+idx] - dir0
    dirs_a_b = dirs_a + dirs_b
    norm_dirs_ab = (dirs_ab.transpose()/np.linalg.norm(dirs_ab, axis=1)).transpose()
    norm_dirs_a_b = (dirs_a_b.transpose()/np.linalg.norm(dirs_a_b, axis=1)).transpose()
    dps = np.sum(norm_dirs_ab * norm_dirs_a_b, axis=1)
    angs = np.arccos(dps)
    avg_ang = angs.sum() / angs.size
    all_avg_angs.append(avg_ang)
    #print("Average angle: %f radians" % (avg_ang))
    
avg_avg_ang = sum(all_avg_angs) / len(all_avg_angs)
print("Average average angle: %f radians" % (avg_avg_ang))
print("Average average angle: %f degrees" % (180. * avg_avg_ang / np.pi))

In [ ]:
# bootstrap test:
# For each pair of muscles choose two individual fields at random,
# compare with their sum
all_avg_angs_list = []
n_smp = 400
for itr in range(n_smp):
    all_avg_angsB = []
    for idx, pair in enumerate(double_stim):
        a = np.random.randint(1,7)
        b = a
        while b == a:
            b = np.random.randint(1,7)
        dirs_a = all_fhc[a] - all_hc[a] - dir0
        dirs_b = all_fhc[b] - all_hc[b] - dir0
        dirs_a_b = dirs_a + dirs_b
        dirs_ab = all_fhc[7+idx] - all_hc[7+idx] - dir0
        norm_dirs_ab = (dirs_ab.transpose()/np.linalg.norm(dirs_ab, axis=1)).transpose()
        norm_dirs_a_b = (dirs_a_b.transpose()/np.linalg.norm(dirs_a_b, axis=1)).transpose()
        dps = np.sum(norm_dirs_ab * norm_dirs_a_b, axis=1)
        angs = np.arccos(dps)
        avg_ang = angs.sum() / angs.size
        all_avg_angsB.append(avg_ang)
    all_avg_angs_list.append(sum(all_avg_angsB) / len(all_avg_angsB))
    
p_val = .001
print("Mean shuffled average average angle: %f radians" % (np.mean(all_avg_angs_list)))
print("Shuffled average average angle: %f degrees" % (180. * np.mean(all_avg_angs_list) / np.pi))
print("Standard deviation of average angles: %f radians" % (np.std(all_avg_angs_list)))
max_avg_for_p = np.percentile(all_avg_angs_list, 100.*p_val)
print("Max average angle that reaches p value: %f" % (max_avg_for_p))
if max_avg_for_p > avg_avg_ang:
    print("AVERAGE SMALLER THAN P VALUE")

In [ ]:
# plot direction fields.
scale = 6. # factor to scale the direction vectors (when using arrow)
m1 = 0 # first muscle to stimulate
m2 = 3 # second muscle to stimulate
quiver = False # use quiver to plot (otherwise arrow is used)
tit_s = 30 # title font size
tic_s = 30 # tick size
lab_s = 25 # label size

# find the index (p_idx) of the (m1,m2) pair in double_stim
if m1 > m2: # make sure m1 is the smallest
    m = m2;    m2 = m1;    m1 = m
elif m1 == m2:
    raise ValueError('m1 == m2')
p_idx = 0
while p_idx < 15:
    if m1 == double_stim[p_idx][0] and m2 == double_stim[p_idx][1]:
        break
    else:
        p_idx += 1

dirs0 = scale * (all_fhc[0] - all_hc[0])
    
#---------------------------------------------------------------------------------------------------------
#--------------------------- Resting field removed -------------------------------------------------------
#---------------------------------------------------------------------------------------------------------
dirs1 = scale * (all_fhc[1+m1] - all_hc[1+m1]) - dirs0
dirs2 = scale * (all_fhc[1+m2] - all_hc[1+m2]) - dirs0
dirs3 = scale * (all_fhc[7+p_idx] - all_hc[7+p_idx]) - dirs0
dirs4 = dirs1 + dirs2

# obtaining the coordinate with the least change for each field
min_coord0 = all_hc[0][np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = all_hc[1+m1][np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = all_hc[1+m2][np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs4, axis=1))]

# optionally, reduce the number of points to plot
skip = 1
all_hc_copy = all_hc.copy()
all_fhc_copy = all_fhc.copy()
all_hc = [hc[::skip] for hc in all_hc]
all_fhc = [fhc[::skip] for fhc in all_fhc]
dirs0 = dirs0[::skip]
dirs1 = dirs1[::skip]
dirs2 = dirs2[::skip]
dirs3 = dirs3[::skip]
dirs4 = dirs4[::skip]

dir_fig, dir_axes = plt.subplots(2,2,figsize=(40,40)) #, sharex=True, sharey=True)
if quiver:
    q1 = dir_axes[0,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q2 = dir_axes[0,1].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q3 = dir_axes[1,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q3 = dir_axes[1,0].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q4 = dir_axes[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs4[:,0], dirs4[:,1], color='blue')
    q4 = dir_axes[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs3[:,0], dirs3[:,1], color='green')
else:
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[7+p_idx], dirs3):
        dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
    for coords, dirs in zip(all_hc[7+p_idx], dirs4):
        dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
        
#scat1 = dir_axes[0,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
dir_axes[0,0].set_title("D("+ str(m1) +"), RF removed", fontsize=tit_s)
dir_axes[0,0].tick_params(axis='both', labelsize=tic_s)
#scat2 = dir_axes[0,1].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[0,1].set_title("D("+ str(m2) +"), RF removed", fontsize=tit_s)
dir_axes[0,1].tick_params(axis='both', labelsize=tic_s)
#scat3 = dir_axes[1,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
#scat3 = dir_axes[1,0].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[1,0].set_title("D("+ str(m1) +"), D("+ str(m2) +") overlapped, RFs removed", fontsize=tit_s)
dir_axes[1,0].tick_params(axis='both', labelsize=tic_s)
scat4 = dir_axes[1,1].scatter(min_coord3[0], min_coord3[1],  s=500, c='green', alpha=0.9)
scat4 = dir_axes[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='blue')
dir_axes[1,1].set_title("D("+ str(m1) +"+"+str(m2) + "), D("+ str(m1) +") + D("+ str(m2) +"), RFs removed", fontsize=tit_s)
dir_axes[1,1].tick_params(axis='both', labelsize=tic_s)
    
for ax in dir_axes.flatten():
    ax.tick_params(axis='both', labelsize=tic_s)
    
# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("Resting field removed:")
print("Average angle: %f radians, %f degrees" % (avg_ang, 180.*avg_ang/np.pi))

all_hc = all_hc_copy
all_fhc = all_fhc_copy

#---------------------------------------------------------------------------------------------------------
#----------------------- Resting field not removed -------------------------------------------------------
#---------------------------------------------------------------------------------------------------------

dirs1 = scale * (all_fhc[1+m1] - all_hc[1+m1])
dirs2 = scale * (all_fhc[1+m2] - all_hc[1+m2])
dirs3 = scale * (all_fhc[7+p_idx] - all_hc[7+p_idx])
dirs4 = dirs1 + dirs2

# obtaining the coordinate with the least change for each field
min_coord0 = all_hc[0][np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = all_hc[1+m1][np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = all_hc[1+m2][np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs4, axis=1))]

# optionally, reduce the number of points to plot
skip = 4
all_hc_copy = all_hc.copy()
all_fhc_copy = all_fhc.copy()
all_hc = [hc[::skip] for hc in all_hc]
all_fhc = [fhc[::skip] for fhc in all_fhc]
dirs0 = dirs0[::skip]
dirs1 = dirs1[::skip]
dirs2 = dirs2[::skip]
dirs3 = dirs3[::skip]
dirs4 = dirs4[::skip]

dir_fig2, dir_axes2 = plt.subplots(2,2,figsize=(40,40)) #, sharex=True, sharey=True)
if quiver:
    q1 = dir_axes2[0,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='blue')
    q2 = dir_axes2[0,1].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q3 = dir_axes2[1,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='blue')
    q3 = dir_axes2[1,0].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q4 = dir_axes2[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs4[:,0], dirs4[:,1], color='tab:purple')
    q4 = dir_axes2[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs3[:,0], dirs3[:,1], color='green')
else:
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes2[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes2[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes2[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes2[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[7+p_idx], dirs3):
        dir_axes2[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='tab:purple')
    for coords, dirs in zip(all_hc[7+p_idx], dirs4):
        dir_axes2[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
        
scat1 = dir_axes2[0,0].scatter(min_coord1[0], min_coord1[1],  s=700, c='blue')
dir_axes2[0,0].set_title("D("+ str(m1) +")", fontsize=tit_s)
dir_axes2[0,0].tick_params(axis='both', labelsize=tic_s)
scat2 = dir_axes2[0,1].scatter(min_coord2[0], min_coord2[1],  s=700, c='red')
dir_axes2[0,1].set_title("D("+ str(m2) +")", fontsize=tit_s)
dir_axes2[0,1].tick_params(axis='both', labelsize=tic_s)
scat3 = dir_axes2[1,0].scatter(min_coord1[0], min_coord1[1],  s=700, c='blue')
scat3 = dir_axes2[1,0].scatter(min_coord2[0], min_coord2[1],  s=700, c='red')
dir_axes2[1,0].set_title("D("+ str(m1) +"), D("+ str(m2) +") overlapped", fontsize=tit_s)
dir_axes2[1,0].tick_params(axis='both', labelsize=tic_s)
scat4 = dir_axes2[1,1].scatter(min_coord3[0], min_coord3[1],  s=700, c='green', alpha=0.9)
scat4 = dir_axes2[1,1].scatter(min_coord4[0], min_coord4[1],  s=700, c='tab:purple')
dir_axes2[1,1].set_title("D("+ str(m1) +"+"+str(m2) + "), D("+ str(m1) +") + D("+ str(m2) +")", fontsize=tit_s)
dir_axes2[1,1].tick_params(axis='both', labelsize=tic_s)

# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("\n Resting field not removed:")
print("Average angle: %f radians, %f degrees" % (avg_ang, 180.*avg_ang/np.pi))

# Plot resting field and targets

r = 0.1
n_trgs = 8
center = np.array([0.3, 0.3]) # initial hand location
angs = np.linspace(0., 2.*np.pi, n_trgs+1)[:-1]
circle = np.array([np.array([np.cos(ang),np.sin(ang)]) for ang in angs])
targets = center + r*circle # coordinates of the targets

dir0_fig = plt.figure(figsize=(15,15))
ax = plt.gca()
if quiver:
    q0 = ax.quiver(all_hc[0][:,0], all_hc[0][:,1], dirs0[:,0], dirs0[:,1], color='blue')
else:
    for coords, dirs in zip(all_hc[0], dirs0):
        ax.arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
scat0 = ax.scatter(min_coord0[0], min_coord0[1],  s=500, c='blue')
ax.scatter(targets[:,0], targets[:,1], s=1800, c='cyan')
ax.plot([center[0]], [center[1]], 'g+', linewidth=5)
ax.scatter([center[0]], [center[1]], s=2000, c='green')
plt.title("Resting Field", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
ax.set_xlim([0.15,.45])
ax.set_ylim([0.15,.45])
ax.set_xlabel('X [m]', fontsize=lab_s)
ax.set_ylabel('Y [m]', fontsize=lab_s)

plt.show()

all_hc = all_hc_copy
all_fhc = all_fhc_copy

In [ ]:
# plot direction fields.
scale = 6. # factor to scale the direction vectors (when using arrow)
m1 = 0 # first muscle to stimulate
m2 = 3 # second muscle to stimulate
quiver = False # use quiver to plot (otherwise arrow is used)
tit_s = 30 # title font size
tic_s = 30 # tick size
lab_s = 25 # label size

if m1 > m2: # make sure m1 is the smallest
    m = m2;    m2 = m1;    m1 = m
elif m1 == m2:
    raise ValueError('m1 == m2')
p_idx = 0
while p_idx < 15:
    if m1 == double_stim[p_idx][0] and m2 == double_stim[p_idx][1]:
        break
    else:
        p_idx += 1

dirs0 = scale * (all_fhc[0] - all_hc[0])
    
#---------------------------------------------------------------------------------------------------------
#--------------------------- Resting field removed -------------------------------------------------------
#---------------------------------------------------------------------------------------------------------
dirs1 = scale * (all_fhc[1+m1] - all_hc[1+m1]) - dirs0
dirs2 = scale * (all_fhc[1+m2] - all_hc[1+m2]) - dirs0
dirs3 = scale * (all_fhc[7+p_idx] - all_hc[7+p_idx]) - dirs0
dirs4 = dirs1 + dirs2

# obtaining the coordinate with the least change for each field
min_coord0 = all_hc[0][np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = all_hc[1+m1][np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = all_hc[1+m2][np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs4, axis=1))]

dir_fig, dir_axes = plt.subplots(2,2,figsize=(40,40)) #, sharex=True, sharey=True)
if quiver:
    q1 = dir_axes[0,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q2 = dir_axes[0,1].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q3 = dir_axes[1,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q3 = dir_axes[1,0].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q4 = dir_axes[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs4[:,0], dirs4[:,1], color='blue')
    q4 = dir_axes[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs3[:,0], dirs3[:,1], color='green')
else:
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[7+p_idx], dirs3):
        dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
    for coords, dirs in zip(all_hc[7+p_idx], dirs4):
        dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
        
#scat1 = dir_axes[0,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
dir_axes[0,0].set_title("D("+ str(m1) +"), RF removed", fontsize=tit_s)
dir_axes[0,0].tick_params(axis='both', labelsize=tic_s)
#scat2 = dir_axes[0,1].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[0,1].set_title("D("+ str(m2) +"), RF removed", fontsize=tit_s)
dir_axes[0,1].tick_params(axis='both', labelsize=tic_s)
#scat3 = dir_axes[1,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
#scat3 = dir_axes[1,0].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[1,0].set_title("D("+ str(m1) +"), D("+ str(m2) +") overlapped, RFs removed", fontsize=tit_s)
dir_axes[1,0].tick_params(axis='both', labelsize=tic_s)
scat4 = dir_axes[1,1].scatter(min_coord3[0], min_coord3[1],  s=500, c='green', alpha=0.9)
scat4 = dir_axes[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='blue')
dir_axes[1,1].set_title("D("+ str(m1) +"+"+str(m2) + "), D("+ str(m1) +") + D("+ str(m2) +"), RFs removed", fontsize=tit_s)
dir_axes[1,1].tick_params(axis='both', labelsize=tic_s)
    
for ax in dir_axes.flatten():
    ax.tick_params(axis='both', labelsize=tic_s)
    
# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("Resting field removed:")
print("Average angle: %f radians, %f degrees" % (avg_ang, 180.*avg_ang/np.pi))

#---------------------------------------------------------------------------------------------------------
#----------------------- Resting field not removed -------------------------------------------------------
#---------------------------------------------------------------------------------------------------------

dirs1 = scale * (all_fhc[1+m1] - all_hc[1+m1])
dirs2 = scale * (all_fhc[1+m2] - all_hc[1+m2])
dirs3 = scale * (all_fhc[7+p_idx] - all_hc[7+p_idx])
dirs4 = dirs1 + dirs2

# obtaining the coordinate with the least change for each field
min_coord0 = all_hc[0][np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = all_hc[1+m1][np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = all_hc[1+m2][np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = all_hc[7+p_idx][np.argmin(np.linalg.norm(dirs4, axis=1))]

dir_fig2, dir_axes2 = plt.subplots(2,2,figsize=(40,40)) #, sharex=True, sharey=True)
if quiver:
    q1 = dir_axes2[0,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q2 = dir_axes2[0,1].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q3 = dir_axes2[1,0].quiver(all_hc[1+m1][:,0], all_hc[1+m1][:,1], dirs1[:,0], dirs1[:,1], color='black')
    q3 = dir_axes2[1,0].quiver(all_hc[1+m2][:,0], all_hc[1+m2][:,1], dirs2[:,0], dirs2[:,1], color='red')
    q4 = dir_axes2[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs4[:,0], dirs4[:,1], color='blue')
    q4 = dir_axes2[1,1].quiver(all_hc[7+p_idx][:,0], all_hc[7+p_idx][:,1], dirs3[:,0], dirs3[:,1], color='green')
else:
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes2[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes2[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[1+m1], dirs1):
        dir_axes2[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='black')
    for coords, dirs in zip(all_hc[1+m2], dirs2):
        dir_axes2[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
    for coords, dirs in zip(all_hc[7+p_idx], dirs3):
        dir_axes2[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
    for coords, dirs in zip(all_hc[7+p_idx], dirs4):
        dir_axes2[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
        
scat1 = dir_axes2[0,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
dir_axes2[0,0].set_title("D("+ str(m1) +")", fontsize=tit_s)
dir_axes2[0,0].tick_params(axis='both', labelsize=tic_s)
scat2 = dir_axes2[0,1].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes2[0,1].set_title("D("+ str(m2) +")", fontsize=tit_s)
dir_axes2[0,1].tick_params(axis='both', labelsize=tic_s)
scat3 = dir_axes2[1,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
scat3 = dir_axes2[1,0].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes2[1,0].set_title("D("+ str(m1) +"), D("+ str(m2) +") overlapped", fontsize=tit_s)
dir_axes2[1,0].tick_params(axis='both', labelsize=tic_s)
scat4 = dir_axes2[1,1].scatter(min_coord3[0], min_coord3[1],  s=500, c='green', alpha=0.9)
scat4 = dir_axes2[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='blue')
dir_axes2[1,1].set_title("D("+ str(m1) +"+"+str(m2) + "), D("+ str(m1) +") + D("+ str(m2) +")", fontsize=tit_s)
dir_axes2[1,1].tick_params(axis='both', labelsize=tic_s)

# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("\n Resting field not removed:")
print("Average angle: %f radians, %f degrees" % (avg_ang, 180.*avg_ang/np.pi))

# Plot resting field and targets

r = 0.1
n_trgs = 8
center = np.array([0.3, 0.3]) # initial hand location
angs = np.linspace(0., 2.*np.pi, n_trgs+1)[:-1]
circle = np.array([np.array([np.cos(ang),np.sin(ang)]) for ang in angs])
targets = center + r*circle # coordinates of the targets

dir0_fig = plt.figure(figsize=(15,15))
ax = plt.gca()
if quiver:
    q0 = ax.quiver(all_hc[0][:,0], all_hc[0][:,1], dirs0[:,0], dirs0[:,1], color='blue')
else:
    for coords, dirs in zip(all_hc[0], dirs0):
        ax.arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
scat0 = ax.scatter(min_coord0[0], min_coord0[1],  s=500, c='blue')
ax.scatter(targets[:,0], targets[:,1], s=1800, c='cyan')
ax.plot([center[0]], [center[1]], 'g+', linewidth=5)
ax.scatter([center[0]], [center[1]], s=2000, c='green')
plt.title("Resting Field", fontsize=tit_s)
plt.xticks(fontsize=tic_s)
plt.yticks(fontsize=tic_s)
ax.set_xlim([0.15,.45])
ax.set_ylim([0.15,.45])
ax.set_xlabel('X [m]', fontsize=lab_s)
ax.set_ylabel('Y [m]', fontsize=lab_s)

plt.show()

---

---

In [ ]:
# run simulations for one set of combinations
start_wctime = time.time()
sim_time = 1. # 1. simulation time
E_amp = 1. # amplitude of stimulus to CE population
I_amp = 1. # amplitude of stimulus to CI population
n_procs = 30 # number of processes to use for grid simulation
# no C units are stimulated (resting field)
hand_coords0, final_hand_coords0, t0, ad0, apd0 = get_dir_field_par([], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs)
# A couple of flexor units stimulated
hand_coords1, final_hand_coords1, t1, ad1, apd1 = get_dir_field_par([0], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs)
# A couple of extensor units stimulated
#hand_coords2, final_hand_coords2, t2, ad2, apd2 = get_dir_field_par([0,1], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs)
hand_coords2, final_hand_coords2, t2, ad2, apd2 = get_dir_field_par([3], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs) # alternative stimulus
# Both flexor and extensor units stimulated
#hand_coords3, final_hand_coords3, t3, ad3, apd3 = get_dir_field_par([0, 1, 2, 3], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs)
hand_coords3, final_hand_coords3, t3, ad3, apd3 = get_dir_field_par([0, 3], sim_time, CE_w=E_amp, CI_w=I_amp, n_procs=n_procs) # alternative stimulus
print("done!")
print('\n Execution time is %s seconds' % (time.time() - start_wctime))

In [ ]:
# plot direction fields.
scale = 4. # factor to scale the direction vectors
dirs0 = scale * (final_hand_coords0 - hand_coords0)
dirs1 = scale * (final_hand_coords1 - hand_coords1)
dirs2 = scale * (final_hand_coords2 - hand_coords2)
dirs3 = scale * (final_hand_coords3 - hand_coords3)
dirs4 = dirs1 + dirs2

# obtaining the coordinate with the least change for each field
min_coord0 = hand_coords0[np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = hand_coords0[np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = hand_coords0[np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = hand_coords0[np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = hand_coords0[np.argmin(np.linalg.norm(dirs4, axis=1))]

dir_fig, dir_axes = plt.subplots(2,2,figsize=(40,40))
#q1 = dir_axes[0,0].quiver(hand_coords1[:,0], hand_coords1[:,1], dirs1[:,0], dirs1[:,1])
for coords, dirs in zip(hand_coords1, dirs1):
    dir_axes[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1])
scat1 = dir_axes[0,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
dir_axes[0,0].set_title("field 1", fontsize=20)

#q2 = dir_axes[0,1].quiver(hand_coords2[:,0], hand_coords2[:,1], dirs2[:,0], dirs2[:,1])
for coords, dirs in zip(hand_coords2, dirs2):
    dir_axes[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
scat2 = dir_axes[0,1].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[0,1].set_title("field 2", fontsize=20)

#q3 = dir_axes[1,0].quiver(hand_coords1[:,0], hand_coords1[:,1], dirs1[:,0], dirs1[:,1])
#q3 = dir_axes[1,0].quiver(hand_coords2[:,0], hand_coords2[:,1], dirs2[:,0], dirs2[:,1], color='red')
for coords, dirs in zip(hand_coords1, dirs1):
    dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1])
for coords, dirs in zip(hand_coords2, dirs2):
    dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
scat3 = dir_axes[1,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
scat3 = dir_axes[1,0].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[1,0].set_title("field 1, field 2 overlapped", fontsize=20)

#q4 = dir_axes[1,1].quiver(hand_coords3[:,0], hand_coords3[:,1], dirs4[:,0], dirs4[:,1], color='blue')
for coords, dirs in zip(hand_coords3, dirs3):
    dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')
for coords, dirs in zip(hand_coords3, dirs4):
    dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
scat4 = dir_axes[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='green')
dir_axes[1,1].set_title("field 1+2, field 1 + field 2", fontsize=20)

dir_fig0 = plt.figure(figsize=(20,20))
ax = plt.gca()
for coords, dirs in zip(hand_coords0, dirs0):
    ax.arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')

plt.show()

# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("Average angle: %f radians" % (avg_ang))
print("Average angle: %f degrees" % (180.*avg_ang/np.pi))

In [ ]:
# plot directionsfields. Version with resting field removed.
scale = 5.
dirs0 = scale * (final_hand_coords0 - hand_coords0)
dirs1 = scale * (final_hand_coords1 - hand_coords1 - dirs0)
dirs2 = scale * (final_hand_coords2 - hand_coords2 - dirs0)
dirs3 = scale * (final_hand_coords3 - hand_coords3 - dirs0)
dirs4 = dirs1 + dirs2
# obtaining the coordinate with the least change for each field
min_coord0 = hand_coords0[np.argmin(np.linalg.norm(dirs0, axis=1))]
min_coord1 = hand_coords0[np.argmin(np.linalg.norm(dirs1, axis=1))]
min_coord2 = hand_coords0[np.argmin(np.linalg.norm(dirs2, axis=1))]
min_coord3 = hand_coords0[np.argmin(np.linalg.norm(dirs3, axis=1))]
min_coord4 = hand_coords0[np.argmin(np.linalg.norm(dirs4, axis=1))]

dir_fig, dir_axes = plt.subplots(2,2,figsize=(40,40))
q1 = dir_axes[0,0].quiver(hand_coords1[:,0], hand_coords1[:,1], dirs1[:,0], dirs1[:,1])
# for coords, dirs in zip(hand_coords1, dirs1):
#     dir_axes[0,0].arrow(coords[0], coords[1], dirs[0], dirs[1])
scat1 = dir_axes[0,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
dir_axes[0,0].set_title("field 1", fontsize=20)

q2 = dir_axes[0,1].quiver(hand_coords2[:,0], hand_coords2[:,1], dirs2[:,0], dirs2[:,1], color='red')
# for coords, dirs in zip(hand_coords2, dirs2):
#     dir_axes[0,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
scat2 = dir_axes[0,1].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[0,1].set_title("field 2", fontsize=20)

q3 = dir_axes[1,0].quiver(hand_coords1[:,0], hand_coords1[:,1], dirs1[:,0], dirs1[:,1], color='black')
q3 = dir_axes[1,0].quiver(hand_coords2[:,0], hand_coords2[:,1], dirs2[:,0], dirs2[:,1], color='red')
# for coords, dirs in zip(hand_coords1, dirs1):
#     dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1])
# for coords, dirs in zip(hand_coords2, dirs2):
#     dir_axes[1,0].arrow(coords[0], coords[1], dirs[0], dirs[1], color='red')
scat3 = dir_axes[1,0].scatter(min_coord1[0], min_coord1[1],  s=500, c='black')
scat3 = dir_axes[1,0].scatter(min_coord2[0], min_coord2[1],  s=500, c='red')
dir_axes[1,0].set_title("field 1, field 2 overlapped", fontsize=20)

q4 = dir_axes[1,1].quiver(hand_coords3[:,0], hand_coords3[:,1], dirs3[:,0], dirs3[:,1], color='pink')
q4 = dir_axes[1,1].quiver(hand_coords3[:,0], hand_coords3[:,1], dirs4[:,0], dirs4[:,1], color='green')
# for coords, dirs in zip(hand_coords3, dirs3):
#     dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='pink')
# for coords, dirs in zip(hand_coords3, dirs4):
#     dir_axes[1,1].arrow(coords[0], coords[1], dirs[0], dirs[1], color='green')
scat4 = dir_axes[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='green')
#scat4 = dir_axes[1,1].scatter(min_coord4[0], min_coord4[1],  s=500, c='green')
dir_axes[1,1].set_title("field 1+2, field 1 + field 2", fontsize=20)

dir_fig0 = plt.figure(figsize=(20,20))
ax = plt.gca()
for coords, dirs in zip(hand_coords0, dirs0):
    ax.arrow(coords[0], coords[1], dirs[0], dirs[1], color='blue')

# obtain a similarity measure of fields 3 and 4 (average angle)
# 1) normalize the direction vectors
norm_dirs3 = np.zeros_like(dirs3)
for idx, vec in enumerate(dirs3):
    norm_dirs3[idx, :] = vec / np.linalg.norm(vec)
norm_dirs4 = np.zeros_like(dirs4)
for idx, vec in enumerate(dirs4):
    norm_dirs4[idx, :] = vec / np.linalg.norm(vec)
# 2) Obtain the dot products between normalized vectors
dps = np.sum(norm_dirs3*norm_dirs4, axis=1)
# 3) Obtain the average angle
angs = np.arccos(dps)
avg_ang = angs.sum()/angs.size
print("Average angle: %f radians" % (avg_ang))
print("Average angle: %f degrees" % (180.*avg_ang/np.pi))

In [ ]:
# plot trajectories

net, pops_dict = create_net([0.3,0.3])
for name in list(pops_dict.keys()):
    exec("%s = %s"% (name, str(pops_dict[name])))

l1 = net.plants[P].l_arm
l2 = net.plants[P].l_farm
apd = [apd1, apd2, apd3, apd0] # the resting field (apd0) is at the end
#apd = [apd0]
# Multiple plots
#----------------
traj_fig, traj_axes = plt.subplots(2,2,figsize=(20,20))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
for idx, all_plant_data in enumerate(apd):
    for pd_id, plant_data in enumerate(all_plant_data):
        # angles of shoulder and elbow
        theta_s = plant_data[:,0]
        theta_e = plant_data[:,2]
        phi = theta_s + theta_e # elbow angle wrt x axis
        # coordinates of hand and elbow    
        xe = np.cos(theta_s)*l1
        ye = np.sin(theta_s)*l1
        xh = xe + np.cos(phi)*l2
        yh = ye + np.sin(phi)*l2
        traj_axes[int(idx/2),idx%2].plot(xh,yh, color=colors[pd_id%5])
        traj_axes[int(idx/2),idx%2].grid()
        traj_axes[int(idx/2),idx%2].scatter(xh[0],yh[0], color=colors[pd_id%5])
    traj_axes[int(idx/2),idx%2].set_title("field " + str(idx))

plt.show()

---
---
---
---

In [ ]:
# Plot fraction of resting length needed to reach each target
r = 0.1
n_trgs = 8
center = np.array([0.3, 0.3]) # initial hand location
angs = np.linspace(0., 2.*np.pi, n_trgs+1)[:-1]
circle = np.array([np.array([np.cos(ang),np.sin(ang)]) for ang in angs])
targets = center + r*circle # coordinates of the targets

# obtain muscle lengths at all targets
m_lengths = np.zeros((6,8))
for tidx, target in enumerate(targets):
    net, pops_dict = create_net(target)
    plant = net.plants[pops_dict['P']]
    plant.upd_ip()
    lengths, speeds = plant.muscle_kinematics(0., 0., plant.c_elbow, plant.ip)
    m_lengths[:,tidx] = lengths

# lengths at center
c_net, c_pops_dict = create_net(center)
c_plant = c_net.plants[c_pops_dict['P']]
c_plant.upd_ip()
c_lengths, _ = c_plant.muscle_kinematics(0., 0., c_plant.c_elbow, c_plant.ip)

# obtain length difference
len_diffs = np.zeros_like(m_lengths)
for trg_id in range(len(targets)):
    len_diffs[:,trg_id] = np.maximum(c_lengths - m_lengths[:,trg_id], 0.) / c_lengths
    #len_diffs[:,trg_id] = (c_lengths - m_lengths[:,trg_id])
    
# normalize lengths
#m_lengths = m_lengths / np.tile(np.linalg.norm(m_lengths, axis=1), (8,1)).transpose()
#m_lengths -= np.tile(np.mean(m_lengths, axis=1), (8,1)).transpose()
#m_lengths -= np.tile(np.min(m_lengths, axis=1), (8,1)).transpose()

# obtain average vectors
m_vecs = np.zeros((6,2))
m_angs = np.zeros(6)
for idx, mlen in enumerate(m_lengths):
    m_vecs[idx,0] = (mlen*np.cos(angs)).sum()
    m_vecs[idx,1] = (mlen*np.sin(angs)).sum()
    m_angs[idx] = np.arctan2(m_vecs[idx,1], m_vecs[idx,0])
m_norms = np.linalg.norm(m_vecs, axis=1)

# predicted direction vectors
ldiff_vecs = np.zeros((6,2)) # predicted direction vectors
ldiff_angs = np.zeros(6)
for idx, lds in enumerate(len_diffs):
    ldiff_vecs[idx,0] = (lds*np.cos(angs)).sum()
    ldiff_vecs[idx,1] = (lds*np.sin(angs)).sum()
    ldiff_angs[idx] = np.arctan2(ldiff_vecs[idx,1], ldiff_vecs[idx,0])
ldiff_norms = np.linalg.norm(ldiff_vecs, axis=1)

# Plot muscle lengths per target
fs = (43,6)
dir_fig, dir_axs = plt.subplots(1, 6, figsize=(fs), subplot_kw=dict(projection='polar'), sharey=True)

#span = 1.5*max(m_lengths.flatten())
theta = np.linspace(0., 2 * np.pi, 8, endpoint=False)
width = np.pi / 8

for col in range(6):
    ax = dir_axs[col]
    m_id = col
    ax.set_title("muscle %d" % (m_id), fontsize=25)
    ax.bar(theta, len_diffs[m_id, :], width=width, bottom=0.0, color='g', alpha=0.3)
    ax.arrow(ldiff_angs[m_id],.01, 0., .5*ldiff_norms[m_id], width=0.07,
        head_width=0.15, head_length=0.02, length_includes_head=True, color='r')
    #ax.bar(theta, m_lengths[m_id, :], width=width, bottom=0.0, color='g', alpha=0.3)
    #ax.arrow(m_angs[m_id],.01, 0., .5*m_norms[m_id], width=0.07,
    #     head_width=0.15, head_length=0.02, length_includes_head=True, color='r')
    ax.tick_params(axis='x', labelsize=22)
    ax.tick_params(axis='y', labelsize=18)
    ax.set_yticks([.05, .1, .15])
    
                    
plt.show()

---
---
---
---